In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset, load_from_disk, Dataset, concatenate_datasets, DatasetDict
import pandas as pd

### 異なる種類のDatasetを集め，統合する
- 仮方針: Datasetのtrain部分から1100データ取得し，train1000, test100に使用する.
- 統合後のDatasetのfeatureとして，メインの入力を`text`, 追加のタグとしてカテゴリの種類を`zoo_tag`として付与
  
  - 元Datasetに`text`が存在しない場合は，入力総統のfeatureを`text`として格納
  - map()を使用して処理

In [35]:
label_CS = 'kunishou/J-ResearchCorpus' # split=[`train`], main_key_name = `text`
label_LEGAL = 'y2lan/japan-law' # split=[`train`], main_key_name = `body`
# label_MED # MMedBenchをDLし，jsonlを直読み
label_NEWS = 'fujiki/llm-japanese-dataset_wikinews' # # split=[`train`], main_key_name = `input`
label_CODE = 'codeparrot/github-code' # split=[`train`], main_key_name = `code`
label_MATH = 'saldra/sakura_japanese_dataset' # split=`train`], main_key_name = `instruction` & `output`
label_Other = 'izumi-lab/mc4-ja'  # split=[`train`, `validation`] main_key_name = `text`


In [36]:
#  ---------------- Corpus/Dataset ダウンロード用関数
def get_corpus(corpus_name_or_path, split, row_num=1000):
  """ 指定されたDatasetから指定した数のdataをlistとして取得 StreamingなのでDatasetすべてをDLしなくて済む　"""
  datas = []
  iterable_dataset = load_dataset(corpus_name_or_path, split=split, streaming=True)
  print(iterable_dataset) # debug
  for data in iterable_dataset.take(row_num):
    datas.append(data)

  return datas

def get_corpus_fullDL(corpus_name_or_path, split, row_num=1000):
  """ TODO: DatasetをすべてDLする. 大規模になりがちなのでまだ未テスト """
  datas = []
  dataset = load_dataset(corpus_name_or_path, split=split)
  print(dataset) # debug
  if row_num > dataset.num_rows:
    print(f'Error corpus_name_or_path len = `{dataset.num_rows}` < {row_num}')
    exit()
  for data in dataset.take(row_num):
    datas.append(data)

  return datas


# ---- Datasetへのmap関数
def add_datasetTag(example, tag_name):
  """ Datasetのitemに key:zoo_tag: tag_name を追加する """
  example['zoo_tag'] = tag_name
  return example

def add_text_as_main_target(example, use_main_key):
  """ Datasetのitemとして，メイン入力扱いするkey=`text`とするため， 元Datasetのfeaturesからkey=`text`とするfeatureを選択し，その値を`text`にもcopy
    元のDatasetに`text`が存在していた場合には，指定されたkeyの値で上書きされることに注意
  """
  example['text'] = example[use_main_key]
  return example

def add_text_as_main_target_use_2feature(example, concat_key1, concat_key2):
  """ textとして，複数のfeatureから構成される場合にテキストとして結合 """
  example['text'] = f"{example[concat_key1]}\n{example[concat_key2]}"
  return example

In [37]:
### CS test
CSpaper_list = get_corpus(label_CS, 'train', 1100)
ds_CSpaper = Dataset.from_dict(pd.DataFrame(data=CSpaper_list))  # jsonのlistからDataset形式で再構築
print(ds_CSpaper[0:2])

ds_CSpaper = ds_CSpaper.map(add_datasetTag, fn_kwargs={'tag_name': "CS"})
print(ds_CSpaper[0])
dsdict_CSpaper = ds_CSpaper.train_test_split(test_size=100)  # train , test 分割

IterableDataset({
    features: Unknown,
    n_shards: 1
})
{'filename': ['A1-1.pdf', 'A1-2.pdf'], 'text': ['# ニューラル機械翻訳における Iterative Back-Translation を利用した コンパラブルコーパスの活用\n\n山本 優紀 秋葉 友良 塚田 元\n豊橋技術科学大学\n\\{yamamoto.yuki.pr, akiba.tomoyoshi.tk, tsukada.hajime.hl\\}@tut.jp\n\n## 概要\n\nニューラル機械翻訳 (NMT) の学習に用いる対訳コーパスの構築法として, 文書単位で対応付けられた 2 つの言語のコーパス (コンパラブルコーパス) から、対応付けられる文ペアを自動的に抽出する手法が広く採用されている. しかし, 文単位で意味が対応するものは少なく,多くの文は抽出されず捨てられてしまう. 本研究では、対訳コーパスとして抽出されなかった文を含めて,コンパラブルコー パス全体を NMT の学習に活用する手法を提案する. 評価実験により, コンパラブルコーパスでデータ拡張を行うことや, コンパラブル性の利用, Iterative Back-Translation の活用によって翻訳モデルの性能が向上することを確認した.\n\n## 1 はじめに\n\n機械翻訳の分野では, 深層学習の発達により, ニューラルネットワークを用いるニューラル機械翻訳 (Neural Machine Translation:NMT) が, 従来手法の統計的機械翻訳よりも高い性能を示しており, 様々な研究が行われている. NMT では, ニューラルネットワークで構築した翻訳モデルを, 翻訳元の言語 (原言語) の文と,その訳の言語 (目的言語) の文のぺアにした対訳コーパスを用いて学習を行う. NMT は, 対訳コーパスから翻訳に関わる様々な知識を学習するため, 対訳コーパスの質や量が NMT モデルの翻訳性能に大きく影響する.しかし, 大規模な対訳コーパスを人手で作成することは困難という問題点がある.\n\nこの問題の解決策として, 既存の日本語と英語の翻訳テキストから対訳コーパスを構築する手法が提案されている.[1]

Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

{'filename': 'A1-1.pdf', 'text': '# ニューラル機械翻訳における Iterative Back-Translation を利用した コンパラブルコーパスの活用\n\n山本 優紀 秋葉 友良 塚田 元\n豊橋技術科学大学\n\\{yamamoto.yuki.pr, akiba.tomoyoshi.tk, tsukada.hajime.hl\\}@tut.jp\n\n## 概要\n\nニューラル機械翻訳 (NMT) の学習に用いる対訳コーパスの構築法として, 文書単位で対応付けられた 2 つの言語のコーパス (コンパラブルコーパス) から、対応付けられる文ペアを自動的に抽出する手法が広く採用されている. しかし, 文単位で意味が対応するものは少なく,多くの文は抽出されず捨てられてしまう. 本研究では、対訳コーパスとして抽出されなかった文を含めて,コンパラブルコー パス全体を NMT の学習に活用する手法を提案する. 評価実験により, コンパラブルコーパスでデータ拡張を行うことや, コンパラブル性の利用, Iterative Back-Translation の活用によって翻訳モデルの性能が向上することを確認した.\n\n## 1 はじめに\n\n機械翻訳の分野では, 深層学習の発達により, ニューラルネットワークを用いるニューラル機械翻訳 (Neural Machine Translation:NMT) が, 従来手法の統計的機械翻訳よりも高い性能を示しており, 様々な研究が行われている. NMT では, ニューラルネットワークで構築した翻訳モデルを, 翻訳元の言語 (原言語) の文と,その訳の言語 (目的言語) の文のぺアにした対訳コーパスを用いて学習を行う. NMT は, 対訳コーパスから翻訳に関わる様々な知識を学習するため, 対訳コーパスの質や量が NMT モデルの翻訳性能に大きく影響する.しかし, 大規模な対訳コーパスを人手で作成することは困難という問題点がある.\n\nこの問題の解決策として, 既存の日本語と英語の翻訳テキストから対訳コーパスを構築する手法が提案されている.[1]これは, 新聞などの文書単位で対応付けつけられた 2 つの言語コーパス (コンパラブルコーパス) から, 対応付けられる文ぺアを自動的\nに抽出するこ

In [38]:
### LEGAL
LEGAL_list = get_corpus(label_LEGAL, 'train', 1100)
ds_LEGAL = Dataset.from_dict(pd.DataFrame(data=LEGAL_list))  # jsonのlistからDataset形式で再構築
ds_LEGAL = ds_LEGAL.map(add_datasetTag, fn_kwargs={'tag_name': "LEGAL"})
ds_LEGAL = ds_LEGAL.map(add_text_as_main_target, fn_kwargs={'use_main_key': 'body'})
print(f'text: {ds_LEGAL[0]["text"][0:200]}\n---\nzoo_tag: {ds_LEGAL[0]["zoo_tag"]}')
dsdict_LEGAL = ds_LEGAL.train_test_split(test_size=100)

IterableDataset({
    features: Unknown,
    n_shards: 1
})


Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

text: 情報通信技術を活用した行政の推進等に関する法律（平成十四年法律第百五十一号）第六条第一項及び第四項から第六項まで、第七条第一項、第四項及び第五項、第八条第一項並びに第九条第一項及び第三項の規定に基づき、並びに同法及び関係法令を実施するため、公害等調整委員会関係法令に係る情報通信技術を活用した行政の推進等に関する法律施行規則を次のように定める。


（趣旨）
第一条


公害等調整委員会関係法令に規
---
zoo_tag: LEGAL


In [39]:
### MED
# https://huggingface.co/datasets/Henrychur/MMedBench (cc-by-4.0) QA形式
# .zipをDLし，その中のJapanese.jsonlを読み込む  load_dataset('Henrychur/MMedBench', 'ja', 'train')  # 指定言語を得られなかった

def get_MMedBench(save_jsonl_path, row_num=1100):
  """ MMedBench の内， 言語.jsonl のパスから, Datasetを取得 `text`にQAの質問-選択肢群-回答　のテキスト結合したものを格納 """
  MED_ds = load_dataset("json", data_files=save_jsonl_path)['train']

  # print 3 example
  for data in MED_ds.select(range(3)):
    for feature in MED_ds[0].keys():
      print(f'`{feature}`\t{data[feature]}')
    print('\n---')

  # Create specified Row NUM Dataset
  MED_list = []
  for data in MED_ds.select(range(row_num)):
    MED_list.append(data)
  ds_MED = Dataset.from_dict(pd.DataFrame(data=MED_list))  # jsonのlistからDataset形式で再構築

  # --- map 関数
  def concatenete_MED_QA2text(example):
    def get_options(options_dict):
      """ 選択肢とその値をまとめたテキストに"""
      option_list = []
      for op_key, op_value in options_dict.items():
        if op_value is None:
          continue
        else:
          option_list.append(f'{op_key}: {op_value}')
      return " ".join(option_list)

    op_text = get_options(example['options'])
    example['text'] = f"{example['question']}\n{op_text}\n{example['answer_idx']}: {example['rationale']}"

    return example

  ds_MED = ds_MED.map(concatenete_MED_QA2text)
  for feature in ds_MED[0].keys():    # test print
    print(f'`{feature}`\t{ds_MED[0][feature]}')

  return ds_MED


save_MED_path = "/content/drive/MyDrive/Colab Notebooks/MyProjects/test_LLM/test_jsonl/MMedBench/Japanese.jsonl"
ds_MED = get_MMedBench(save_MED_path, 1100)
ds_MED = ds_MED.map(add_datasetTag, fn_kwargs={'tag_name': "MED"})
print(len(ds_MED))
dsdict_MED = ds_MED.train_test_split(test_size=100)

`problem_id`	116E20
`question`	労働安全衛生法に規定される健康の保持増進のための措置について誤りはどれか。
`options`	{'A': 'うつ病や自殺の予防が目的に含まれる。', 'B': '適用となるのは常時50人以上の事業場である。', 'C': '脳血管・心臓疾患のリスク管理として重要である。', 'D': '時間外労働が月80時間超の労働者は希望により面接指導が受けられる。', 'E': '事業者は過重労働者に対し医師による面接指導の実施が義務づけられている。', 'F': None, 'G': None, 'H': None}
`answer_idx`	['B']
`rationale`	選択肢B「適用となるのは常時50人以上の事業場である」が誤りです。労働安全衛生法において健康の保持増進のための措置は、事業場の規模に関係なく、すべての労働者に適用されます。法律は事業場の規模による区別を設けておらず、常時50人以上の事業場に限定されるわけではないため、このような規定は存在しません。従って、他の選択肢A, C, D, およびEは労働安全衛生法における健康の保持増進の措置を正しく示しており、うつ病や自殺の予防、脳血管・心臓疾患のリスク管理、時間外労働の監視や過重労働者への医師による面接指導の実施などが含まれています。
`human_checked`	0
`human_check_passed`	0

---
`problem_id`	113F45
`question`	救急外来で小児を診察した研修医から指導医への報告を示す。　研修医：「1歳の女児です。3日前から発熱、咳嗽、鼻汁が続き、本日から四肢、体幹に発疹が出現したため来院しました。咳嗽がひどくルームエアーでSpO2が92％であり、入院も考慮する必要があると思います」　指導医：「どのような発疹ですか」　研修医：「四肢、体幹に紅色の丘疹がひろがっています」　指導医：「口腔内の所見はどうですか」　研修医：「咽頭発赤があり、頬粘膜に白い斑点があります」　指導医：「入院させる場合、この患児で特に必要な感染対策は何ですか」これに続く研修医の返答として最も適切なのはどれか。
`options`	{'A': '「カーテンで隔離を行います」', 'B': '「聴診器を患児専用にします」',

Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

`problem_id`	116E20
`question`	労働安全衛生法に規定される健康の保持増進のための措置について誤りはどれか。
`options`	{'A': 'うつ病や自殺の予防が目的に含まれる。', 'B': '適用となるのは常時50人以上の事業場である。', 'C': '脳血管・心臓疾患のリスク管理として重要である。', 'D': '時間外労働が月80時間超の労働者は希望により面接指導が受けられる。', 'E': '事業者は過重労働者に対し医師による面接指導の実施が義務づけられている。', 'F': None, 'G': None, 'H': None}
`answer_idx`	['B']
`rationale`	選択肢B「適用となるのは常時50人以上の事業場である」が誤りです。労働安全衛生法において健康の保持増進のための措置は、事業場の規模に関係なく、すべての労働者に適用されます。法律は事業場の規模による区別を設けておらず、常時50人以上の事業場に限定されるわけではないため、このような規定は存在しません。従って、他の選択肢A, C, D, およびEは労働安全衛生法における健康の保持増進の措置を正しく示しており、うつ病や自殺の予防、脳血管・心臓疾患のリスク管理、時間外労働の監視や過重労働者への医師による面接指導の実施などが含まれています。
`human_checked`	0
`human_check_passed`	0
`text`	労働安全衛生法に規定される健康の保持増進のための措置について誤りはどれか。
A: うつ病や自殺の予防が目的に含まれる。 B: 適用となるのは常時50人以上の事業場である。 C: 脳血管・心臓疾患のリスク管理として重要である。 D: 時間外労働が月80時間超の労働者は希望により面接指導が受けられる。 E: 事業者は過重労働者に対し医師による面接指導の実施が義務づけられている。
['B']: 選択肢B「適用となるのは常時50人以上の事業場である」が誤りです。労働安全衛生法において健康の保持増進のための措置は、事業場の規模に関係なく、すべての労働者に適用されます。法律は事業場の規模による区別を設けておらず、常時50人以上の事業場に限定されるわけではないため、このような規定は存在しません。従って、他の選択肢A, C, D, お

Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

1100


In [40]:
### REALNEWS
NEWS_list = get_corpus(label_NEWS, 'train', 1100)
ds_NEWS = Dataset.from_dict(pd.DataFrame(data=NEWS_list))
ds_NEWS = ds_NEWS.map(add_datasetTag, fn_kwargs={'tag_name': "NEWS"})
ds_NEWS = ds_NEWS.map(add_text_as_main_target, fn_kwargs={'use_main_key': 'input'})
print(ds_NEWS[0]['zoo_tag'])
print(ds_NEWS[0]['text'])
dsdict_NEWS = ds_NEWS.train_test_split(test_size=100)

IterableDataset({
    features: ['instruction', 'input', 'output'],
    n_shards: 1
})


Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

NEWS
台湾の自由時報によると、3日、陳水扁総統夫人の呉淑珍氏と馬永成元総統府副秘書長、林徳訓総統府秘書らが国務機密費の流用容疑で、台北地方検察院により起訴された。台北地検は、総統夫人を主犯、総統府秘書であった馬氏、林氏の両名が共犯と認定した。読売新聞によれば、呉氏は2002年7月から2006年3月までの間に、機密費1,480万台湾ドル（日本円で約5,300万円）を横領し、指輪の購入など私的な目的に流用した疑いが持たれている。台北地検は陳総統本人も文書を偽造するなどして関与したと認定しているが、在任中の総統であるため、起訴することができなかった。これに対して、与党の民進党は呉総統夫人の起訴について中央評議会で審議するとともに、陳総統に対して国民の前で説明するよう要求した。自由時報によれば、野党の中国国民党と親民党は48時間以内に陳総統が辞任する事を求めた。もしそれが受け入れられなければ3回目の総統罷免案を立法院で動議する方針である。また民進党と政治的主張が近い台湾団結聯盟も、今回の罷免案に賛成する意向である。台湾団結聯盟の精神的指導者である李登輝・前総統も、12月9日に予定されている台北・高雄両直轄市の市長・市議会議員選挙において民進党候補を支援しないと述べている。一方、西日本新聞によれば、陳総統は5日夜、テレビで、「私的流用はなく、横領ではない」と述べ、容疑を否定した。また、「一審で有罪判決ならば総統を辞任する」と述べた。


In [41]:
### CODING
CODE_list = get_corpus(label_CODE, 'train', 1100)
ds_CODE = Dataset.from_dict(pd.DataFrame(data=CODE_list))  # jsonのlistからDataset形式で再構築
ds_CODE = ds_CODE.map(add_datasetTag, fn_kwargs={'tag_name': "CODE"})
ds_CODE = ds_CODE.map(add_text_as_main_target, fn_kwargs={'use_main_key': 'code'})
print(f'text: {ds_CODE[0]["text"][0:200]}\n---\nzoo_tag: {ds_CODE[0]["zoo_tag"]}')
dsdict_CODE = ds_CODE.train_test_split(test_size=100)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for codeparrot/github-code contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/codeparrot/github-code
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


IterableDataset({
    features: ['code', 'repo_name', 'path', 'language', 'license', 'size'],
    n_shards: 1126
})


Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

text: 'use strict';

var clear          = require('es5-ext/array/#/clear')
  , eIndexOf       = require('es5-ext/array/#/e-index-of')
  , setPrototypeOf = require('es5-ext/object/set-prototype-of')
  , call
---
zoo_tag: CODE


In [42]:
### MATH  (row=500)
MATH_list = get_corpus(label_MATH, 'train', 500)
print(len(MATH_list))
ds_MATH = Dataset.from_dict(pd.DataFrame(data=MATH_list))  # jsonのlistからDataset形式で再構築
ds_MATH = ds_MATH.map(add_datasetTag, fn_kwargs={'tag_name': "MATH"})
ds_MATH = ds_MATH.map(add_text_as_main_target_use_2feature, fn_kwargs={'concat_key1': 'instruction', 'concat_key2': 'output'})
print(f'text: {ds_MATH[0]["text"][0:200]}\n---\nzoo_tag: {ds_MATH[0]["zoo_tag"]}')
dsdict_MATH = ds_MATH.train_test_split(test_size=100)

IterableDataset({
    features: Unknown,
    n_shards: 1
})
500


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

text: 5年生は自主的に本の寄付活動に参加しました。 5-1組は500冊、5-2組は5-1組が寄付した本の80％、5-3組は5-2組が寄付した本の120％を寄付しました。 5-1組と5-3組ではどちらが多く本を寄付したでしょうか？ (2通りで比較してください)。
x=1 よって答えは 1組
---
zoo_tag: MATH


In [43]:
### 1B=Other (その他)
#Other_list = get_mc4('train', 1100)
Other_list = get_corpus(label_Other, 'train', 1100)
print(len(Other_list))
ds_Other = Dataset.from_dict(pd.DataFrame(data=Other_list))
ds_Other = ds_Other.map(add_datasetTag, fn_kwargs={'tag_name': "Other"})

print(f'text: {ds_Other[0]["text"][0:200]}\n---\nzoo_tag: {ds_Other[0]["zoo_tag"]}')
dsdict_Other = ds_Other.train_test_split(test_size=100)

Resolving data files:   0%|          | 0/416 [00:00<?, ?it/s]

IterableDataset({
    features: ['text', 'timestamp', 'url'],
    n_shards: 416
})
1100


Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

text: 生八つ橋のタグまとめ | エキサイトブログ
生八つ橋のタグまとめ
「生八つ橋」のタグがついている新着記事と人気記事をまとめました。エキサイトブログには生八つ橋に関連するブログ（日記、記録、写真、レビュー、噂、まとめ）がたくさん投稿されています。
「生八つ橋」タグの記事（4）
生八つ橋いろいろ
京都旅行のお土産(我が家用)に色々な生八つ橋を買ってきました。我が家はみんな八つ橋ファンなのです～。地元の
---
zoo_tag: Other


In [44]:
### Dataset結合
def clean_dataset(dataset, keep_features):
  # remove unnecessaly features except specified keep_features
  remove_features = list(set(dataset.features) - set(keep_features))
  ds = dataset.remove_columns(remove_features)
  return ds

dsdict_list = [dsdict_CSpaper, dsdict_LEGAL, dsdict_MED, dsdict_NEWS, dsdict_CODE, dsdict_MATH, dsdict_Other]
train_ds = concatenate_datasets([clean_dataset(ds['train'], keep_features=['text', 'zoo_tag']) for ds in dsdict_list])
print(train_ds)
print(set(train_ds['zoo_tag'][:]))

test_ds = concatenate_datasets([clean_dataset(ds['test'], keep_features=['text', 'zoo_tag']) for ds in dsdict_list])
print(test_ds)
print(set(test_ds['zoo_tag'][:]))

# DatasetDictとしてまとめる
combined_datasetDict = DatasetDict({"train": train_ds, "test": test_ds})
print(combined_datasetDict)

Dataset({
    features: ['text', 'zoo_tag'],
    num_rows: 6400
})
{'NEWS', 'CS', 'Other', 'MATH', 'CODE', 'LEGAL', 'MED'}
Dataset({
    features: ['text', 'zoo_tag'],
    num_rows: 700
})
{'NEWS', 'CS', 'Other', 'MATH', 'CODE', 'LEGAL', 'MED'}
DatasetDict({
    train: Dataset({
        features: ['text', 'zoo_tag'],
        num_rows: 6400
    })
    test: Dataset({
        features: ['text', 'zoo_tag'],
        num_rows: 700
    })
})


In [45]:
### Save & Load Dataset / DatasetDict
# Prepare Save Path
import os
from google.colab import drive
drive.mount('/content/drive')
!ls


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive  sample_data


In [46]:
save_datasetDict_path = "/content/drive/MyDrive/Colab Notebooks/MyProjects/test_LLM/test_jsonl/"  # TODO 各自環境に書き換え
save_DirName = "mix_7categories_dataset"

combined_datasetDict.save_to_disk(save_datasetDict_path + save_DirName)
print('saved')
loaded_datasetDict = load_from_disk(save_datasetDict_path + save_DirName)
print(loaded_datasetDict)

Saving the dataset (0/1 shards):   0%|          | 0/6400 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/700 [00:00<?, ? examples/s]

saved
DatasetDict({
    train: Dataset({
        features: ['text', 'zoo_tag'],
        num_rows: 6400
    })
    test: Dataset({
        features: ['text', 'zoo_tag'],
        num_rows: 700
    })
})
